reduccion de columnas a las necesarias

In [1]:
%%bigquery

CREATE OR REPLACE TABLE `canvas-sum-435121-c0.dataset_procesado.Metadata_Google_new` AS
SELECT
    gmap_id,
    name,
    address,
    description,
    latitude,
    longitude,
    category,
    num_of_reviews
FROM
    `canvas-sum-435121-c0.Dataset_California.Metadata_Google`
WHERE
   latitude BETWEEN 32.5 AND 42.0
   AND longitude BETWEEN -124.5 AND -114.0;



Query is running:   0%|          |

""


In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `canvas-sum-435121-c0.dataset_procesado.Reviews_Google_new` AS
SELECT
    name,
    rating,
    text,
    resp,
    gmap_id
FROM
    `canvas-sum-435121-c0.Dataset_California.Reviews_Google`;

Query is running:   0%|          |

""


In [6]:
%%bigquery

CREATE OR REPLACE TABLE `canvas-sum-435121-c0.dataset_procesado.Yelp_Reviews_California_new` AS
SELECT
    business_id,
    name,
    city,
    state,
    postal_code,
    latitude,
    longitude,
    stars_x,
    review_count,
    categories,
    review_id,
    stars_y,
    useful,
    funny,
    cool,
    text
FROM
    `canvas-sum-435121-c0.Dataset_California.Yelp_Reviews_California`
WHERE
     categories LIKE '%Restaurant%';



Query is running:   0%|          |

""


modelo de entidad relacion

crear union entre Metadata_google y reviews_google

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `canvas-sum-435121-c0.entidad_relacion.Reviews_Google` AS
SELECT
    m.gmap_id,
    m.name,
    m.address,
    m.latitude,
    m.longitude,
    r.name AS name_user,   -- Renombrar el campo name a name_user
    r.rating,
    r.text AS review_text,  -- Renombrar text si es necesario
    r.resp,
FROM
    `canvas-sum-435121-c0.dataset_procesado.Metadata_Google_new` AS m
JOIN
    `canvas-sum-435121-c0.dataset_procesado.Reviews_Google_new` AS r
ON
    m.gmap_id = r.gmap_id  -- Suponiendo que se une por gmap_id
WHERE
    m.gmap_id IS NOT NULL
    AND r.gmap_id IS NOT NULL;




Query is running:   0%|          |

""


Filtrar Entradas para Mantener Solo los Registros No Duplicados:

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `canvas-sum-435121-c0.entidad_relacion.Location` AS
WITH GoogleData AS (
    SELECT
        FORMAT('%f,%f', g.latitude, g.longitude) AS unique_location,
        g.name AS google_name,
        g.latitude,
        g.longitude,
        g.address AS google_address
    FROM
        `canvas-sum-435121-c0.entidad_relacion.Reviews_Google` g
),
YelpData AS (
    SELECT
        FORMAT('%f,%f', y.latitude, y.longitude) AS unique_location,
        y.name AS yelp_name,
        y.categories AS category,
        y.latitude,
        y.longitude,
        y.postal_code,
        y.city
    FROM
        `canvas-sum-435121-c0.entidad_relacion.Yelp_Reviews` y
)

-- Unimos los datos de Google y Yelp en base a latitud y longitud
SELECT
    ROW_NUMBER() OVER() AS location_id,
    COALESCE(g.google_name, y.yelp_name) AS name,       -- Usar el nombre de Google o Yelp
    y.category,                                         -- Categoría de Yelp
    g.latitude,
    g.longitude,
    g.google_address,                                   -- Dirección de Google
    y.postal_code,                                      -- Código postal de Yelp
    y.city                                              -- Ciudad de Yelp
FROM
    GoogleData g
LEFT JOIN
    YelpData y
ON
    g.unique_location = y.unique_location;


Query is running:   0%|          |

""


In [ ]:
%%bigquery

-- Actualizar Reviews_Google: añadir location_id y eliminar columnas redundantes
CREATE OR REPLACE TABLE `canvas-sum-435121-c0.entidad_relacion.Reviews_Google_new` AS
SELECT
    r.gmap_id,  -- Mantener el ID del google
    r.name_user,
    r.rating,
    r.review_text,
    r.resp,
    l.location_id  -- Añadir el nuevo identificador de ubicación
FROM
    `canvas-sum-435121-c0.entidad_relacion.Reviews_Google` r
JOIN
    `canvas-sum-435121-c0.entidad_relacion.Location` l
ON
    r.latitude = l.latitude
    AND r.longitude = l.longitude;



Query is running:   0%|          |

""


In [ ]:
%%bigquery

-- Actualizar Yelp_Reviews: añadir location_id y eliminar columnas redundantes
CREATE OR REPLACE TABLE `canvas-sum-435121-c0.entidad_relacion.Yelp_Reviews_new` AS
SELECT
    y.review_id,  -- Mantener el ID del review
    y.stars_x AS stars_business,
    y.stars_y AS stars_reviews,
    y.text,
    y.useful,
    y.funny,
    y.cool,
    l.location_id  -- Añadir el nuevo identificador de ubicación
FROM
    `canvas-sum-435121-c0.entidad_relacion.Yelp_Reviews` y
JOIN
    `canvas-sum-435121-c0.entidad_relacion.Location` l
ON
    y.latitude = l.latitude
    AND y.longitude = l.longitude;


Executing query with job ID: 9c9345ff-aaa9-499d-a20e-3456d0062c6f
Query executing: 0.40s


ERROR:
 400 Name latitude not found inside y at [17:7]; reason: invalidQuery, location: query, message: Name latitude not found inside y at [17:7]

Location: us-central1
Job ID: 9c9345ff-aaa9-499d-a20e-3456d0062c6f

